## Embedding + Aggregation

this notebook generate students vector in "course" with CodeBook k (k:the number of centroids, 100 is default)
".\data\vectors\norm_Student_Vctors_course{course}_{k}dim.csv"

In [190]:
import pandas as pd
import numpy as np
import fasttext as ft
import re
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

Using CodeBook and Students Actions 
Genarating Students Vectors

select courses and k(the number of centroids)

In [191]:
course= "D-2021"
k=100

In [192]:
Code_Book_dir = r".\data\code_book"
CodeBook_file = Code_Book_dir + r"\CodeBook_k{}.csv".format(k)

### prepareing for Generating Student vectors in "course"

actions : action files on one course  

output : dataframe of each students action

In [193]:
actions = r".\data\actions_txt\actions_{}_perStudents.txt".format(course)
users_actions = []
user_action = ""
user=[]
with open(actions, "r") as f:
    actions_list = f.readlines()
    for action in actions_list:
        if action.startswith("****"):
            user.append(action.rstrip("\n").replace("*",""))
            users_actions.append(user_action)
            user_action = ""
        else:
            user_action += action + " "
user_df = pd.DataFrame(users_actions)
user_df["userid"] = user
user_df.set_index("userid",inplace=True)

In [194]:
user_df

,0
userid,
D-2021_U1,OsNmNs PsNl\n PmN\n OsNNPsNmPsNNm NPsN\n Om NN...
D-2021_U2,OmNmPmNsNsNm NNsNm OsNmCm OmNmNs PNmNNmNmNs NN...
D-2021_U3,ONNsNNmPsPPm PsNNPm Nl\n NsN\n OsNNNNNNNPPNNN_...
D-2021_U4,ONNsPNPsPNmNmN_ NmNmNsNsNsNsNs_ NNNsNNsNNsNsNN...
D-2021_U5,OsNNl\n C\n ONsNsNsNmNm NsNmNNsPsNsNsNm_ NsNsP...
...,...
D-2021_U104,OsNsNsNsNsPPl\n Nm Nl\n C\n OsNmNmPNm Pl\n Nm ...
D-2021_U105,OsNmPNsPsNNmPs_ NsNmPPmNPsNN NsNmNsPsNsNNNs_ P...
D-2021_U9,OmNPNsNsNsNsNs_ NmNmNNNNNNsNNN_ NsPNNNNNNNsPsN...


### Aggregation

In [195]:
## read CodeBook
action_centroids = pd.read_csv(CodeBook_file,index_col=0)
## load fastText model (we must use the same model to Making CodeBook)
model = ft.load_model(r".\fastText_ALL-2020_paperused.bin")


### Aggregation step
# users_vecs: output students vector 
users_vecs =[]
# users actions
users_data = user_df.values
# aggregation step for one student
for actions_one_user in users_data:
        # user_vec = one student aggregated vector 
        user_vec = [0*i for i in range(len(action_centroids))]
        # split each actions 
        actions_oneuser = actions_one_user[0].split("\n") 
        for one_action in actions_oneuser:
            # if empty action, skip vectorization
            if one_action == "" or one_action == " ":
                    continue
            # vector of action generated by fastText
            # embedding
            one_action_vec = model.get_sentence_vector(one_action)
            # calculate cosine similarity between action vec and centroids
            sim_mat = cosine_similarity(one_action_vec.reshape(1,-1), action_centroids.values)
            # get the most similar centroids ID
            max_index = np.argmax(sim_mat)
            # count the most similar centroids 
            user_vec[max_index] += 1
        users_vecs.append(user_vec)
boa_df = pd.DataFrame(users_vecs,index=user_df.index)

#normarization
norm_boa_df = pd.DataFrame(normalize(boa_df,norm="l2",axis=1),index=boa_df.index)
norm_boa_df.to_csv(r".\data\vectors\norm_Student_Vctors_course{}_{}dim.csv".format(course,k))

In [196]:
norm_boa_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
userid,,,,,,,,,,,,,,,,,,,,,
D-2021_U1,0.310460,0.077615,0.077615,0.038808,0.194038,0.077615,0.038808,0.038808,0.038808,0.000000,...,0.077615,0.194038,0.232845,0.000000,0.000000,0.000000,0.038808,0.077615,0.077615,0.0
D-2021_U2,0.000000,0.000000,0.557086,0.185695,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.185695,0.000000,0.000000,0.000000,0.000000,0.185695,0.185695,0.000000,0.000000,0.0
D-2021_U3,0.196537,0.131024,0.098268,0.000000,0.032756,0.065512,0.032756,0.000000,0.000000,0.131024,...,0.065512,0.262049,0.131024,0.000000,0.032756,0.098268,0.000000,0.000000,0.000000,0.0
D-2021_U4,0.138343,0.000000,0.000000,0.069171,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.069171,0.207514,0.000000,0.000000,0.069171,0.000000,0.000000,0.000000,0.000000,0.0
D-2021_U5,0.000000,0.000000,0.000000,0.000000,0.084215,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.252646,0.000000,0.000000,0.000000,0.000000,0.084215,0.084215,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
D-2021_U104,0.184604,0.046151,0.138453,0.023076,0.069227,0.023076,0.023076,0.000000,0.000000,0.000000,...,0.023076,0.299982,0.184604,0.046151,0.023076,0.023076,0.023076,0.046151,0.161529,0.0
D-2021_U105,0.280056,0.000000,0.000000,0.000000,0.140028,0.000000,0.140028,0.000000,0.000000,0.000000,...,0.000000,0.140028,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
D-2021_U9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.0
